#### https://www.kaggle.com/zalando-research/fashionmnist

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import fashion_mnist

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
#Loading the Fashion Mnist dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)
X_train.shape

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784, )))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
model.summary()

In [ ]:
%%time
history = model.fit(X_train, y_train, epochs=25, batch_size=64,
                    validation_split=0.2, verbose = 0, workers = 5)

In [ ]:
print(history.history.keys())

In [ ]:
# summarize history for loss

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper left')
plt.show()

In [ ]:
# summarize history for accuracy

plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper left')
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

In [ ]:
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights("fashion_model.h5")

In [ ]:
import cv2
for i in range(10):
    cv2.imwrite("uploads/{}.png".format(i), X_test[i])


In [ ]:
import os
import requests

from flask import Flask, request, jsonify

with open('fashion_model.json', 'r') as f:
    model_json = f.read()

model = tf.keras.models.model_from_json(model_json)
model.load_weights("fashion_model.h5")

app = Flask(__name__)

@app.route("/app/<string:img_name>", methods=["POST"]) # Dynamic URL
def classify(img_name):
    upload_dir = "uploads/"
    image = cv2.imread(upload_dir + img_name)
    classes = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", 
               "Shirt", "Sneaker", "Bag", "Ankle boot"]

    prediction = model.predict([image.reshape(1, 28*28).astype("uint8")])

    return jsonify({"object":classes[np.argmax(prediction[0])]})

app.run(port=8000, debug=True, use_reloader=False)